# Basics of Deep Learning Modelling

## Why Julia?

During this course we will work with [Julia](https://julialang.org/) language. Why?

- It is a script language (just like Python or R)
- Julia is fast (almost like C)
- It have developed [type system](https://upload.wikimedia.org/wikipedia/commons/d/d9/Julia-number-type-hierarchy.svg) 
- Build-in functions for distributed computing and GPU usage.
- Julia is easy to integrate with other languages (Python, R, C, …) 


### Additional materials 

- [Julia Academy](https://juliaacademy.com/)
- [Boyd and Vandenberghe Linear Algebra book](http://vmls-book.stanford.edu/)
- [Julia Express](https://github.com/bkamins/The-Julia-Express)
- [Quantitative Economics lectures by Sargent and Stachursky](https://lectures.quantecon.org/jl/)
- [Julia for Data Science](http://ucidatascienceinitiative.github.io/IntroToJulia/)
- [Think Julia](https://benlauwens.github.io/ThinkJulia.jl/latest/book.html)
- [Other materials avalaible at the language site](https://julialang.org/learning/)

### Packages

####  1. DataFrames.jl

[<tt>DataFrames</tt>](https://dataframes.juliadata.org/stable/) is a package with resources for working with tabular data. It is an implementation of well-known data frames, with all the same tools as in the <tt>R</tt> or <tt>Pandas</tt> library in <tt>Python</tt>. If you want to learn more you could take a look at the [introduction to <tt>DataFrames</tt> ](https://github.com/bkamins/Julia-DataFrames-Tutorial).

#### 2. Plots.jl

[Plots](https://docs.juliaplots.org/latest/tutorial/) is a basic tool for plotting in Julia. The biggest advantage of this package is an access to many different plotting [backends](http://docs.juliaplots.org/latest/backends/). Documentation of <tt>Plots.jl</tt> is avalaible [here](http://docs.juliaplots.org/latest/).

# Introductory  Example

Let start with a simple case study. We will build a basic perceptron for the classification task on the Australian credit scoring data (avalaible [here](http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/australian/australian.dat)). Our goal is to train a model to distinguish between the good and bad debtors. Obviously, we must start with importing the necessary packages and preparing a data:

In [ ]:
using DelimitedFiles
using PyPlot
using Statistics
using StatsBase
using DataFrames

In [ ]:
isfile("australian.dat") ||
 download("http://archive.ics.uci.edu/ml/machine-learning-databases/
    statlog/australian/australian.dat")
rawdata = readdlm("australian.dat");

In [ ]:
df = DataFrames.DataFrame(rawdata)
rename!(df,:x15 => :class)
df[!,:x4] = [x == 1 ? 1.0 : 0.0 for x in df[!,:x4]]
df[!,:x12] = [x == 1 ? 1.0 : 0.0 for x in df[!,:x12]]
df[!,:x14] = log.(df[!,:x14])
first(df,5)

In [ ]:
describe(df)

In [ ]:
countmap(df[!, :class])

In [ ]:
train_ratio = 0.7

In [ ]:
train_set = df[1:floor(Int,size(df,1)*train_ratio),:];
test_set = df[floor(Int,size(df,1)*train_ratio + 1):end,:];

In [ ]:
X_train = Matrix(train_set[:,1:end-1])';
X_test = Matrix(test_set[:,1:end-1])';
y_train = train_set[!, :class];
y_test = test_set[!, :class];

Data should be normalized:

In [ ]:
function scale(X)

    μ = mean(X, dims=2)
    σ = std(X, dims=2)

    X_norm = (X .- μ) ./ σ

    return (X_norm, μ, σ);
end

function scale(X, μ, σ)
    X_norm = (X .- μ) ./ σ
    return X_norm;
end

In [ ]:
X_train, μ, σ = scale(X_train);

In [ ]:
X_test = scale(X_test, μ, σ);

Finally we must define the weights of the model and the sigmoidal function:

In [ ]:
β = rand(1,size(X_train,1) + 1);

In [ ]:
Predict(β, x) = 1 ./ (1 .+ exp.(-β[1:end-1]' * x .- β[end]))

In [ ]:
Predict(β,X_train)

<b> Binary cross-entropy </b> (also known as <b>log-loss</b>) is the loss function we will use. It is defined as:

$$ H_p(q) = - \sum_{i=1}^N {y_i log(p(y_i)) + (1 - y_i) log(p(1 -y_i))}$$

In [ ]:
L(ŷ, y) = (-y') * log.(ŷ') - (1 .- y') * log.(1 .- ŷ')

To optimize the weights $\beta$ we will use the simple [gradient descent method](https://en.wikipedia.org/wiki/Gradient_descent):

In [ ]:
function simple_∇(β, X, y)
    J = L(Predict(β, X),y)[1] 
    ∇ = Float64[]
    for i = 1:length(β)
        b = β[i]
        β′ = β .+ (LinearIndices(β) .== i) * b * √eps()
        β′′ = β .- (LinearIndices(β) .== i) * b * √eps()
        Δf = (L(Predict(β′,X),y)[1] - L(Predict(β′′,X),y)[1]) / (2*b*√eps())
        push!(∇,Δf)
    end
    return J, ∇
end

In [ ]:
simple_∇(β,X_train,y_train)

In [ ]:
function solve!(β, X, y;
            η = 0.001, ϵ = 10^-10, maxit = 50_000)
    iter = 1
    Js = Float64[]
    J, ∇ = simple_∇(β, X, y)
    push!(Js,J)
    while true
        β₀ = β
        β -= η * ∇'
        J, ∇ = simple_∇(β, X, y)
        push!(Js,J)
        stop = maximum(abs.(β .- β₀))
        stop < ϵ && break
        iter += 1
        iter > maxit && break
    end
    return Js
end

In [ ]:
Js = solve!(β,X_train, y_train);

In [ ]:
plot(Js)

In [ ]:
accuracy(β, X, y, T = 0.5) = sum((Predict(β, X)' .≥ T ).== y)/length(y)

In [ ]:
accuracy(β, X_test, y_test)

# Deep Learning Models

Obviously we will not define entire models in every case. Especially when models will became much more complicated than the simple perceptron. We will use [flux.jl](http://fluxml.ai/):

- [Flux](http://fluxml.ai/) is a Julia machine learning stack.
- Flux is lightweight, written entirely in Julia; it is trivial to hack it and build models suited for very specific cases. 
- Flux supports all the Julia's syntax; the vast majority of Julia's functions and macros could be used inside the model.
- Building of basic models is very easy and intuitive.

### Layers

Flux allows to define the layer on neural network in many different ways:

In [ ]:
@time using Flux

In [ ]:
W = rand(4, 8)
b = rand(4)
layer₁(x) = 1.0 ./ (1.0.+exp.(-W*x - b))

In [ ]:
W

In [ ]:
x = rand(8)
layer₁(x)

We could also use the already implemented, most popular [types of layers](https://fluxml.ai/Flux.jl/stable/models/layers/#Basic-Layers-1):

In [ ]:
layer₂(x) = σ.(W * x .+ b)
layer₂(x)

In [ ]:
layer₃ = Dense(8,4,σ)
layer₃(x)

We could also define them on our own:

In [ ]:
struct Poly
    W
    V
    b
end

Poly(in::Integer, out::Integer) =
  Poly((randn(out, in)),randn(out, in), (randn(out)))

# Overload call, so the object can be used as a function
(m::Poly)(x) = m.W * x.^2 + m.V*x .+ m.b

a = Poly(10, 5)

a(rand(10)) # => 5-element vector

And then, to use such user-defined layer in the Flux, we must use the macro <tt>@functor </tt>. Without that we will not be able to use all the built-in functions, e.g.  gradient propagation or [GPU computing](https://fluxml.ai/Flux.jl/stable/gpu/):

In [ ]:
Flux.@functor  Poly

In [ ]:
gpu(a);

Obviously, our model will have more than one layer. Again, there are plenty of methods to do this:

In [ ]:
Layer₁ = Dense(28^2, 32, relu)
Layer₂ = Dense(32, 10)
Layer₃ = softmax

Function <tt>Chain</tt> allows us to merge multiple layers in the sequence:

In [ ]:
chain = Chain(x -> x^2, x-> -x)
m₁ = Chain(Layer₁ , Layer₂, Layer₃) 

We could also define the model as a function composition:

In [ ]:
m₂(x) = Layer₃(Layer₂(Layer₁(x)))

In [ ]:
m₃(x) = Layer₁ ∘ Layer₂ ∘ Layer₃  

or pipeline:

In [ ]:
m₄(x) = Layer₁(x) |> Layer₂  |> Layer₃ 

### Cost Function; Regularization

[Goodfellow I., Bengio Y., Courville A. (2016), Deep Learning, Chapter 7](http://www.deeplearningbook.org/contents/regularization.html)

As it was mention in the last lecture, we cannot optimize the neural network directly; we must define and use a cost function $J(\theta)$.

We could define it on our own:

In [ ]:
model = Dense(5,2)
x, y = rand(5), rand(2);
loss(ŷ, y) = sum((ŷ.- y).^2)/ length(y)
loss(model(x), y) 

or use the one [implemented in Flux](https://github.com/FluxML/Flux.jl/blob/8f73dc6e148eedd11463571a0a8215fd87e7e05b/src/layers/stateless.jl):

In [ ]:
Flux.mse(model(x),y)

Well trained model will have the smallest possible <b>generalization error</b>:

[![](https://cdn-images-1.medium.com/max/1600/1*1woqrqfRwmS1xXYHKPMUDw.png)](https://buzzrobot.com/bias-and-variance-11d8e1fee627)


However, neural networks tends to overfit easily. In order to avoid that, we must use a proper <b>regularization</b> method. With a fine-tuned model we could overcome this problem and significantly reduce the error.

The most common methods are:

<b>penalization of the coefficients</b>:

One of the most common methods of regularization; It limit the capacity of the model to overfit by adding a parameter norm penalty in a form: 
     
$\tilde{J}(\theta) = J(\theta) + \alpha\Omega(\theta)$

Two most common methods are:
- $\Omega(\theta) = ||\theta||_1 = \sum_i{|\theta_i|}$     (<i>LASSO</i>, <i>$L_1$ regularization</i>)
- $\Omega(\theta) = ||\theta||_2^2 = \sum_i{\theta_i^2}$ (<i>Tikhonov regularization</i>, <i>Ridge regression</i>, <i>$L_2$ regularization</i>)

They implementation is [as follows](https://fluxml.ai/Flux.jl/stable/models/regularisation/):

In [ ]:
using LinearAlgebra

In [ ]:
L₁(θ) = sum(abs, θ) 
L₂(θ) = sum(abs2, θ) 

In [ ]:
J(x,y,W) = loss(model(x),y) + L₁(W)

In [ ]:
J(x,y,W)

<b>Bagging (bootstrap aggregating)</b>:

From the initial dataset, $k$ sets are sampled uniformly with replacement. Then $k$  models are trained using the above sets. Final result is obtained by aggregating the output of the models.

<b>Dropout</b>:

In every iteration of the training, neurons are removed from the model with a probability $p$. Binary vector $\mu = [1,1,0,1,1,1,\dots,0,1]$ represent the neurons used in the training process in $i$-th iteration. The goal of the training process is to minimize the value of $E_\mu[J(\theta,\mu)]$ for every iteration. As a result, we obtain the unbiased estimator of a gradient function without the costly process of generating and training $k$ separated models.

Dropout is implemented as a [layer of model](https://fluxml.ai/Flux.jl/stable/models/layers/#Normalisation-and-Regularisation-1): 

In [ ]:
model = Chain(Dense(28^2, 32, relu),
    Dropout(0.1),
Dense(32, 10),
BatchNorm(64, relu),
softmax)

### Optimization

[Goodfellow I., Bengio Y., Courville A. (2016), Deep Learning, Chapter 8](http://www.deeplearningbook.org/contents/optimization.html)

Choice of the proper optimization algorithm is the most important step during the neural network training. Cost function minimization is non-trivial task; there are plenty of serious problems which might derail the learning process:
- ill-conditioned Hessian matrix.
- local minimas, plateaus, etc.
- vanishing and exploding gradients

As a result vanilla stochastic gradient method might not be able to solve such problem. There are many modifications of SGD algorithm introduced to overcome these issues:
- SGD [(Robbins & Munro 1951)](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586)
- SGD with momentum [(Polyak, 1964)](http://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid=zvmmf&paperid=7713&option_lang=eng)
- SGD with  Nesterov momentum ([Nesterov, 1983](http://www.cis.pku.edu.cn/faculty/vision/zlin/1983-A%20Method%20of%20Solving%20a%20Convex%20Programming%20Problem%20with%20Convergence%20Rate%20O%28k%5E%28-2%29%29_Nesterov.pdf), [2005](https://www.math.ucdavis.edu/~sqma/MAT258A_Files/Nesterov-2005.pdf))
- AdaGrad (Adaptive Gradient Algorithm) [(Duchi et. al. 2011)](http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf)
- ADAM (Adaptive Moment Estimation) [(Kingma & Ba, 2015)](https://arxiv.org/abs/1412.6980)



Flux allows to [calculate a gradient of every function](https://fluxml.ai/Flux.jl/stable/models/basics/):

In [ ]:
f(x) = 3x^2 + 2x + 1

# df/dx = 6x + 2
df(x) = gradient(f, x)[1]

df(2) # 14.0 

# d²f/dx² = 6
d²f(x) = gradient(df, x)[1]

d²f(2) # 6.0 

Even if the function is not declared as a mathematical formula:

In [ ]:
function pow(x, n)
    r = 1
    for i = 1:n
        r *= x
    end
    return r
end

In [ ]:
pow(2,4)

In [ ]:
gradient(x -> pow(x, 3), 5)

In [ ]:
pow2(x, n) = n <= 0 ? 1 : x*pow2(x, n-1)

In [ ]:
gradient(x -> pow2(x, 3), 5)

It is possible because of the efficient differentiation algorithm implemented in the [<tt>Zygote.jl</tt>](https://fluxml.ai/Zygote.jl/latest/) package. It uses the characteristic elements of the language, e.g. its compiler to find the derivatives of functions. A brief explanation how <tt>Zygote</tt> works is avalaible [here](https://github.com/MikeInnes/diff-zoo) and [here](https://arxiv.org/pdf/1810.07951.pdf).

### Automatic Differentiation

The key element of the neural networks training is finding a gradients. As we now from the previous lecture, naïve numerical approximation of derivatives by the definition:
$$\frac{df}{dx} = \lim_{h \to 0}\frac{f(x_0 +h) - f(x_0)}{h}$$
is a recipe for disaster. So, how we could find the derivatives on the computer?

It turns out, that even the most complicated function could be represented as a composition of some basic functions (sin,cos,log,etc.) and arithmetic operations. Knowing the derivatives of these basic functions, we could compute basically every derivative by using a <i>chain rule</i>:

$$
\frac{dy}{dx} = \frac{dy_1}{dx}*\frac{dy_2}{dy_1}*\dots*\frac{dy_{n-1}}{dy_{n-2}}*\frac{dy}{dy_{n-1}}
$$

There are two distinct methods of automatic differentiation:

- <b>Forward Accumulation</b> we start with a known value of $\frac{dy_0}{dx} = \frac{dx}{dx} = 1$. then we are computing the value of the next step $\frac{dy_1}{dx} = \frac{d_1}{dx} = 1$ and we continue this process, calculating the values of next elements of chain:$\frac{dy_{i+1}}{dy_i}$, until we reach a final step.

- <b>Backward Accumulation</b> we start with a known value of  $\frac{dy}{dy_n} = \frac{dy}{dy} = 1$, then we are finding the value of: $\frac{dy}{dy_n}$, $\frac{dy}{dy_{n-1}}$, ... $\frac{dy}{dy_1}$, $\frac{dy}{dx}$.  

## Zygote.jl

<tt>Zygote</tt> package is based around two crucial elements: macro <tt>@adjoint</tt> and function <tt>pullback</tt>. 

<tt>pullback</tt> returns two things, the value of original function $y = f(x)$ and <i>pullback</i> expression $ \mathcal{B}(\overline{y}) = \overline{y}  \frac{dy}{dx}$, where $\overline{y} = \frac{dl}{dy}$ is a predefined parameter for a given function $l$.  

In [ ]:
using Zygote

In [ ]:
y, back = Zygote.pullback(sin, π);

In [ ]:
y

In [27]:
sin(y)

1.2246467991473532e-16

In [ ]:
back(1)

In particular, for function <tt>gradient</tt>  $l = y = f(x)$ and  $\overline{y} = \frac{dy}{dl} = 1$:

In [ ]:
back(1)

In [ ]:
gradient(sin,π) == back(1)

Macro <tt>@adjoint</tt> allows us to compute custom adjoints and modify the derivation mechanism:

In [ ]:
using Zygote: @adjoint

In [ ]:
minus(a,b) = a - b

In [ ]:
gradient(minus,2,3)

In [ ]:
minus2(a,b) = a - b

In [ ]:
@adjoint minus2(a,b) = minus2(a,b), c̄ -> (nothing, -b^2)

In [ ]:
gradient(minus2,2,3)

 Obviously Flux have implemented [the most common optimization algorithms](https://fluxml.ai/Flux.jl/stable/training/optimisers/):

In [ ]:
opt = ADAM(0.0001)

### Model Training

Flux can control the entire learning process with a function <tt>train!</tt>:

In [ ]:
Flux.train!(objective, data, opt)

However, <tt>train!</tt> works for only one epoch. If we need to train the model for more than one epoch, we must either modify the dataset:

In [ ]:
using Base.Iterators: repeated
dataset = repeated((x, y), 200)

or use the macro <tt>@epochs</tt>:

In [ ]:
Flux.@epochs 2 println("hello")

In flux we could also define the callbacks, which will help us control the learning process:

In [ ]:
evalcb = () -> @show(loss(tX, tY))

## Example

In [ ]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

It is time to wrap everything up. Let start with preparing a dataset:

In [ ]:

# Classify MNIST digits with a simple multi-layer-perceptron

imgs = MNIST.images()
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...) 

labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9) 

Then we must define the model:

In [ ]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) 

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()


accuracy(X, Y)

It is time for some training:

In [ ]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

We could save results in the [<tt>BSON</tt>](https://github.com/JuliaIO/BSON.jl) format:

In [ ]:
using BSON

In [ ]:
BSON.@save "MNIST.bson" m

and obviosly load:

In [ ]:
BSON.@load "MNIST.bson" m

Finally, we could take the look at the results:

In [ ]:
# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) 
tY = onehotbatch(MNIST.labels(:test), 0:9) 

accuracy(tX, tY)

### Hyperparameters Tuning

During the training process, only the weights $\theta$ are optimized. All the other parameters of the neural network (activation functions, regularization methods, optimization algorithm and its parameters, learning rate, etc.) are predefined. In order to find the best model, we must carefully choose them. We might do this with a plenty of different methods:
- using the examples from the literature
- randomly generating and comparing different combinations of hyperparameters
- building a model able to [learn the best parameters for our task](https://arxiv.org/abs/2004.05439)
- searching the hyperparameters space

## Extra Homework

1. Tune the hyperparameters of the network presented during the lecture; try to find the combination which will give you the highest accuracy <b>(5 points)</b>.
2. During the lecture we indtroduce the <i>dual numbers</i>.We define them as expressions in the form: $z = a + \epsilon b$, where $a,b \in \mathbb{R}$ and  $\epsilon^2 = 0$. For every polynomial: $f(x) = a_0 + a_1x + a_2x^2 + \dots + a_nx^n$ its value for the dual number $z$ is equal to: $f(z) = f(a) + bf'(a)\epsilon$, prove it <b>(5 points)</b>.